# Análisis de segmentos posteriores a fase 2

Este cuaderno aborda dos tareas sobre los hipnogramas (**scor_clean**):

1. **Segmentos previos a la salida de fase 2**: contar cuántas épocas consecutivas en fase 2 aparecen inmediatamente después de cada transición proveniente de cualquier fase distinta de 2.
2. **Segmentos en la fase destino**: medir cuánto dura la fase destino justo después de salir de una fase 2, diferenciando cada tipo de transición **2 → X**.

Todos los resultados se generan por paciente y también sobre el conjunto agregado de pacientes, sin utilizar gráficas para mantener la interpretación simple.


## Plan general

1. Importar utilidades y definir rutas.
2. Cargar hipnogramas (**scor_clean**) para cada paciente con archivo **_2**.
3. Para cada paciente:
   - Detectar todas las transiciones **no-2 → 2** y medir la longitud del bloque continuo de 2.
   - Guardar las longitudes en un vector y resumir las frecuencias.
4. Para cada transición **2 → X**, medir cuánto tiempo permanece la nueva fase **X** inmediatamente después de salir de fase 2.
5. Repetir los pasos anteriores de forma agregada con todos los pacientes.
6. Mostrar tablas y estadísticas descriptivas.


In [10]:
from pathlib import Path
from collections import defaultdict

import numpy as np
import pandas as pd
import plotly.express as px
from IPython.display import display

RUTA_DATOS = Path("../Datos")
ETIQUETAS_FASES = {
    0: "Vigilia (W)",
    1: "N1",
    2: "N2",
    3: "N3",
    4: "N4",
    5: "REM",
    6: "Movimiento",
    7: "Sin clasificar"
}

pd.options.display.float_format = lambda x: f"{x:0.3f}"

## Utilidades de carga de hipnogramas


In [11]:
def listar_pacientes(ruta_datos=None):
    """Devuelve los IDs disponibles (archivos `Scoring_*_2`) en orden alfabético."""
    if ruta_datos is None:
        ruta_datos = RUTA_DATOS
    pacientes = []
    for archivo in ruta_datos.iterdir():
        nombre = archivo.name
        if archivo.is_file() and nombre.startswith("Scoring_") and nombre.endswith("_2"):
            pacientes.append(nombre.replace("Scoring_", ""))
    pacientes.sort()
    if not pacientes:
        raise RuntimeError("No se encontraron archivos `Scoring_*_2` en la ruta indicada.")
    return pacientes


def cargar_hipnograma(paciente, ruta_datos=None):
    """Carga la columna `scor_clean` completa del paciente indicado como una lista de enteros."""
    if ruta_datos is None:
        ruta_datos = RUTA_DATOS
    ruta_archivo = ruta_datos / f"Scoring_{paciente}"
    df = pd.read_csv(ruta_archivo, sep=r"\s+", header=None, names=["epoca", "scor_clean", "scor_quasi"], dtype=int)
    return df["scor_clean"].tolist()


pacientes = listar_pacientes()
print(f"Pacientes detectados ({len(pacientes)}): {', '.join(pacientes)}")

Pacientes detectados (10): AR_2, DG_2, EL_2, GA_2, IN_2, JS_2, LL_2, SS_2, VB_2, VC_2


## Tarea 1 — Longitudes de fase 2 tras transiciones **no-2 → 2**

Para cada paciente:
- Revisamos la secuencia **scor_clean** y, cada vez que aparezca un bloque de 2 precedido por una fase diferente de 2, medimos cuántas épocas consecutivas dura ese bloque.
- Guardamos todas las longitudes en un vector y presentamos una tabla de frecuencias para interpretar los tamaños más comunes.
- El mismo análisis se repite con todos los pacientes concatenados para obtener la distribución global.


In [12]:
def extraer_segmentos_fase2(secuencia):
    """Identifica cada bloque continuo de N2 inmediatamente después de una transición no-2 → 2."""
    longitudes = []
    for indice in range(1, len(secuencia)):
        if secuencia[indice] == 2 and secuencia[indice - 1] != 2:
            longitud = 1  # ya contamos el primer 2
            cursor = indice + 1
            while cursor < len(secuencia) and secuencia[cursor] == 2:
                longitud += 1
                cursor += 1
            longitudes.append(longitud)
    return longitudes


def resumen_longitudes(longitudes):
    """Construye una tabla con la frecuencia y el porcentaje de cada longitud observada."""
    if not longitudes:
        return pd.DataFrame(columns=["Longitud", "Frecuencia", "Porcentaje"])
    serie = pd.Series(longitudes)
    conteo = serie.value_counts().sort_index()
    porcentaje = (conteo / conteo.sum()).rename("Porcentaje")
    tabla = pd.concat([conteo.rename("Frecuencia"), porcentaje], axis=1)
    tabla.index.name = "Longitud"
    return tabla.reset_index().round(3)


segmentos_fase2_por_paciente = {}  # paciente → lista de duraciones de N2
for paciente in pacientes:
    hipnograma = cargar_hipnograma(paciente)
    segmentos = extraer_segmentos_fase2(hipnograma)
    segmentos_fase2_por_paciente[paciente] = segmentos
    print(f"\nPaciente {paciente} — segmentos detectados: {len(segmentos)}")
    tabla = resumen_longitudes(segmentos)
    display(tabla.style.format(precision=3).hide(axis="index"))


Paciente AR_2 — segmentos detectados: 46


Longitud,Frecuencia,Porcentaje
1,12,0.261
3,1,0.022
4,1,0.022
5,1,0.022
6,2,0.043
7,2,0.043
8,4,0.087
9,3,0.065
10,1,0.022
11,5,0.109



Paciente DG_2 — segmentos detectados: 57


Longitud,Frecuencia,Porcentaje
1,13,0.228
2,10,0.175
3,9,0.158
4,1,0.018
5,3,0.053
6,2,0.035
7,1,0.018
8,2,0.035
10,2,0.035
11,3,0.053



Paciente EL_2 — segmentos detectados: 52


...

Longitud,Frecuencia,Porcentaje
1,9,0.184
2,8,0.163
3,3,0.061
4,3,0.061
5,3,0.061
6,4,0.082
7,2,0.041
8,2,0.041
10,3,0.061
11,3,0.061



Paciente VB_2 — segmentos detectados: 41


Longitud,Frecuencia,Porcentaje
1,9,0.220
2,7,0.171
3,4,0.098
4,1,0.024
5,3,0.073
7,1,0.024
8,2,0.049
9,1,0.024
11,1,0.024
13,1,0.024



Paciente VC_2 — segmentos detectados: 37


Longitud,Frecuencia,Porcentaje
1,5,0.135
2,6,0.162
3,3,0.081
4,2,0.054
6,2,0.054
8,1,0.027
9,2,0.054
10,3,0.081
13,1,0.027
14,2,0.054


### Cómo leer estas tablas (Tarea 1)

- **Longitud** representa la cantidad de épocas consecutivas en N2 inmediatamente después de la transición **no-2 → 2**.
- **Frecuencia** indica cuántas veces se observó ese tamaño de bloque para el paciente (o para todos los pacientes en el resumen global).
- **Porcentaje** es la proporción respecto al total de transiciones detectadas para ese contexto. Estos valores permiten identificar si, por ejemplo, las transiciones suelen estabilizarse por 1 minuto (2 épocas) o si existen segmentos de N2 mucho más largos.



In [13]:
segmentos_globales = [longitud for lista in segmentos_fase2_por_paciente.values() for longitud in lista]
print(f"\nTotal de segmentos `no-2 → 2` en todos los pacientes: {len(segmentos_globales)}")
tabla_global = resumen_longitudes(segmentos_globales)
display(tabla_global.style.format(precision=3).hide(axis="index"))


Total de segmentos `no-2 → 2` en todos los pacientes: 504


Longitud,Frecuencia,Porcentaje
1,121,0.240
2,59,0.117
3,37,0.073
4,29,0.058
5,30,0.060
6,14,0.028
7,18,0.036
8,20,0.040
9,18,0.036
10,20,0.040


### Interpretación de las tablas (Tarea 2)

- **Fase destino / Etiqueta** identifica el estado al que se llega tras abandonar N2.
- **Segmentos** muestra cuántas veces ocurrió la transición **2 → X** con datos suficientes para medir la duración de la fase destino.
- **Promedio / Mínimo / Máximo** cuantifican cuántas épocas consecutivas se mantuvo la fase destino justo después del cambio. Estos valores ayudan a decidir qué transiciones generan ventanas largas o cortas y cuáles podrían ser candidatas para análisis con embeddings de n-gramas.



## Tarea 2 — Duración de la fase destino tras la salida de fase 2

Objetivo: después de medir cuánto dura cada bloque en fase 2, ahora observamos a qué fase se transita y cuánto tiempo dura esa nueva fase inmediatamente después del cambio.

Para cada transición **2 → X**:
- Identificamos la fase destino **X**.
- Medimos la longitud continua del bloque **X** inmediatamente posterior.
- Guardamos las longitudes en un diccionario **fase destino → [segmentos]**.

La salida consiste en tablas por paciente y un resumen global con estadísticas básicas para cada fase destino.

In [14]:
def extraer_segmentos_destino(secuencia):
    """Agrupa la duración de cada fase destino inmediatamente después de salir de N2."""
    segmentos_por_destino = defaultdict(list)
    indice = 0
    while indice < len(secuencia):
        if secuencia[indice] != 2:
            indice += 1
            continue
        # Entramos a un bloque de 2
        while indice < len(secuencia) and secuencia[indice] == 2:
            indice += 1
        fin = indice  # primer índice distinto de 2 después del bloque
        if fin >= len(secuencia):
            break  # no hay destino porque el hipnograma terminó en 2
        destino = secuencia[fin]
        longitud_destino = 1
        cursor = fin + 1
        while cursor < len(secuencia) and secuencia[cursor] == destino:
            longitud_destino += 1
            cursor += 1
        segmentos_por_destino[destino].append(longitud_destino)
        indice = fin  # continuar evaluando desde la nueva fase
    return segmentos_por_destino


def resumen_destinos(segmentos_por_destino):
    """Genera una tabla con estadísticas básicas para cada transición 2 → X."""
    filas = []
    for destino, longitudes in sorted(segmentos_por_destino.items()):
        if not longitudes:
            continue
        serie = pd.Series(longitudes)
        desviacion = float(serie.std(ddof=0)) if len(longitudes) > 1 else 0.0
        filas.append({
            "Fase destino": destino,
            "Etiqueta": ETIQUETAS_FASES.get(destino, str(destino)),
            "Segmentos": len(longitudes),
            "Promedio": round(serie.mean(), 3),
            "Mediana": round(serie.median(), 3),
            "Desviación": round(desviacion, 3),
            "Mínimo": int(serie.min()),
            "Máximo": int(serie.max())
        })
    columnas = ["Fase destino", "Etiqueta", "Segmentos", "Promedio", "Mediana", "Desviación", "Mínimo", "Máximo"]
    return pd.DataFrame(filas, columns=columnas)


segmentos_destino_por_paciente = {}  # paciente → {fase destino: lista de duraciones}
for paciente in pacientes:
    hipnograma = cargar_hipnograma(paciente)
    destinos = extraer_segmentos_destino(hipnograma)
    segmentos_destino_por_paciente[paciente] = destinos
    print(f"\nPaciente {paciente} — transiciones 2 → X detectadas: {sum(len(v) for v in destinos.values())}")
    tabla_destino = resumen_destinos(destinos)
    display(tabla_destino.style.format(precision=3).hide(axis="index"))


Paciente AR_2 — transiciones 2 → X detectadas: 46


Fase destino,Etiqueta,Segmentos,Promedio,Mediana,Desviación,Mínimo,Máximo
1,N1,2,1.000,1.000,0.000,1,1
3,N3,20,1.550,1.000,1.023,1,5
4,N4,1,3.000,3.000,0.000,3,3
5,REM,7,33.000,34.000,23.934,2,72
6,Movimiento,16,1.500,1.000,1.458,1,7



Paciente DG_2 — transiciones 2 → X detectadas: 57


Fase destino,Etiqueta,Segmentos,Promedio,Mediana,Desviación,Mínimo,Máximo
0,Vigilia (W),3,1.000,1.000,0.000,1,1
1,N1,8,1.125,1.000,0.331,1,2
3,N3,26,1.462,1.000,0.843,1,4
5,REM,7,7.286,8.000,4.589,1,14
6,Movimiento,13,1.462,1.000,0.746,1,3



Paciente EL_2 — transiciones 2 → X detectadas: 52


...

Fase destino,Etiqueta,Segmentos,Promedio,Mediana,Desviación,Mínimo,Máximo
0,Vigilia (W),5,2.200,2.000,0.980,1,4
1,N1,1,1.000,1.000,0.000,1,1
3,N3,25,2.120,1.000,1.925,1,9
4,N4,1,5.000,5.000,0.000,5,5
5,REM,4,31.500,31.000,8.529,23,41
6,Movimiento,13,1.154,1.000,0.361,1,2



Paciente VB_2 — transiciones 2 → X detectadas: 41


Fase destino,Etiqueta,Segmentos,Promedio,Mediana,Desviación,Mínimo,Máximo
0,Vigilia (W),8,4.375,3.500,4.357,1,15
1,N1,3,2.333,2.000,1.247,1,4
3,N3,14,1.714,1.000,1.436,1,6
4,N4,1,1.000,1.000,0.000,1,1
5,REM,9,13.222,10.000,14.289,1,52
6,Movimiento,6,1.000,1.000,0.000,1,1



Paciente VC_2 — transiciones 2 → X detectadas: 37


Fase destino,Etiqueta,Segmentos,Promedio,Mediana,Desviación,Mínimo,Máximo
0,Vigilia (W),2,1.500,1.500,0.500,1,2
3,N3,14,1.500,1.000,0.824,1,4
5,REM,10,19.800,11.000,17.826,3,57
6,Movimiento,11,1.000,1.000,0.000,1,1


In [15]:
segmentos_destino_global = defaultdict(list)
for destinos in segmentos_destino_por_paciente.values():
    for fase_destino, longitudes in destinos.items():
        segmentos_destino_global[fase_destino].extend(longitudes)

print("\nResumen global de fases destino después de salir de 2:")
tabla_destino_global = resumen_destinos(segmentos_destino_global)
display(tabla_destino_global.style.format(precision=3).hide(axis="index"))


Resumen global de fases destino después de salir de 2:


Fase destino,Etiqueta,Segmentos,Promedio,Mediana,Desviación,Mínimo,Máximo
0,Vigilia (W),26,2.538,1.500,2.832,1,15
1,N1,33,1.394,1.000,0.919,1,5
3,N3,223,1.659,1.000,1.267,1,9
4,N4,13,4.154,2.000,4.347,1,17
5,REM,74,19.730,12.000,18.701,1,74
6,Movimiento,135,1.252,1.000,0.685,1,7
7,Sin clasificar,1,1.000,1.000,0.000,1,1


### Datos preparados para análisis posterior

- **segmentos_fase2_por_paciente**: diccionario **paciente → lista de longitudes** con las duraciones de fase 2 posteriores a cada transición **no-2 → 2**.
- **segmentos_globales**: vector con todas las longitudes anteriores combinadas.
- **segmentos_destino_por_paciente**: diccionario **paciente → {fase destino → lista de longitudes}** que describe cuánto tiempo dura cada fase destino tras salir de fase 2.
- **segmentos_destino_global**: agregación de las mismas transiciones para todos los pacientes.

Estos vectores podrán alimentar visualizaciones (box plots) o filtros sobre n-gramas específicos en siguientes iteraciones.


## Boxplots interactivos

Estas visualizaciones completan las dos tareas sin perder la interpretación numérica: permiten revisar la dispersión de las duraciones por paciente y comparar fácilmente qué tan estables son los segmentos de N2 y las fases destino tras abandonar la fase 2. Todas las gráficas usan Plotly con fondo negro, como solicitaste.

In [16]:
def construir_dataframe_segmentos(diccionario):
    """Convierte el diccionario paciente → lista de duraciones en un DataFrame largo."""
    filas = []
    for paciente, valores in diccionario.items():
        for duracion in valores:
            filas.append({
                "Paciente": paciente,
                "Duración N2": duracion
            })
    return pd.DataFrame(filas)


def construir_dataframe_destinos(diccionario):
    """Devuelve un DataFrame con cada duración de la fase destino para boxplots."""
    filas = []
    for paciente, destinos in diccionario.items():
        for fase_destino, longitudes in destinos.items():
            etiqueta = ETIQUETAS_FASES.get(fase_destino, str(fase_destino))
            for duracion in longitudes:
                filas.append({
                    "Paciente": paciente,
                    "Fase destino": fase_destino,
                    "Etiqueta": etiqueta,
                    "Duración destino": duracion
                })
    return pd.DataFrame(filas)

In [17]:
df_segmentos_box = construir_dataframe_segmentos(segmentos_fase2_por_paciente)

if df_segmentos_box.empty:
    print("No hay datos suficientes para graficar las duraciones de N2.")
else:
    fig_pacientes = px.box(
        df_segmentos_box,
        x="Paciente",
        y="Duración N2",
        color="Paciente",
        points="outliers",
        title="Distribución de las duraciones de N2 tras transiciones no-2 → 2",
        template="plotly_dark"
    )
    fig_pacientes.update_layout(
        xaxis_title="Paciente",
        yaxis_title="Duración en N2 (épocas)",
        paper_bgcolor="black",
        plot_bgcolor="black",
        legend_title_text="Paciente",
        font=dict(color="white")
    )
    fig_pacientes.show()

    fig_global = px.box(
        df_segmentos_box,
        y="Duración N2",
        points="outliers",
        title="Distribución global de las duraciones de N2 tras transiciones no-2 → 2",
        template="plotly_dark"
    )
    fig_global.update_layout(
        xaxis_title="",
        yaxis_title="Duración en N2 (épocas)",
        showlegend=False,
        paper_bgcolor="black",
        plot_bgcolor="black",
        font=dict(color="white")
    )
    fig_global.show()

In [18]:
df_destinos_box = construir_dataframe_destinos(segmentos_destino_por_paciente)

if df_destinos_box.empty:
    print("No hay datos suficientes para graficar las fases destino.")
else:
    fig_destinos = px.box(
        df_destinos_box,
        x="Etiqueta",
        y="Duración destino",
        color="Etiqueta",
        points="outliers",
        title="Duración observada tras transitar de N2 a cada fase destino",
        template="plotly_dark"
    )
    fig_destinos.update_layout(
        xaxis_title="Fase destino",
        yaxis_title="Duración en la fase destino (épocas)",
        paper_bgcolor="black",
        plot_bgcolor="black",
        legend_title_text="Fase",
        font=dict(color="white")
    )
    fig_destinos.show()

    fig_destinos_pacientes = px.box(
        df_destinos_box,
        x="Paciente",
        y="Duración destino",
        color="Etiqueta",
        points="outliers",
        title="Duración de la fase destino por paciente",
        template="plotly_dark"
    )
    fig_destinos_pacientes.update_layout(
        xaxis_title="Paciente",
        yaxis_title="Duración en la fase destino (épocas)",
        paper_bgcolor="black",
        plot_bgcolor="black",
        legend_title_text="Fase destino",
        font=dict(color="white")
    )
    fig_destinos_pacientes.show()